<a href="https://colab.research.google.com/github/auchanda/machinelearning/blob/main/butterfly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import cv2 as cv
import warnings
warnings.filterwarnings('ignore')


# In[ ]:


df_train = pd.read_csv("/content/drive/MyDrive/mlearn/dataset/butterfly/Training_set.csv")
df_test = pd.read_csv("/content/drive/MyDrive/mlearn/dataset/butterfly/Testing_set.csv")
df_train.sample(10)
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import cv2 as cv
import warnings
warnings.filterwarnings('ignore')


# In[ ]:


df_train = pd.read_csv("/content/drive/MyDrive/mlearn/dataset/butterfly/Training_set.csv")
df_test = pd.read_csv("/content/drive/MyDrive/mlearn/dataset/butterfly/Testing_set.csv")
df_train.sample(10)


# In[ ]:


df_test.head()


# In[9]:


df_train.info(),df_train.shape


# In[10]:


df_test.info(),df_test.shape


# In[14]:


df = pd.concat([df_train[['filename']],df_test[['filename']]],axis='rows')
df['label'] = [1]*len(df)
df.sample(10)


# In[18]:


from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df_train['label_en'] = encoder.fit_transform(df_train['label'])
df_train.sample(3)


# In[ ]:


# adding non butterfly images
non_target = []
MAIN_FILE = "/content/drive/MyDrive/mlearn/dataset/butterfly/train"
for img in os.listdir(MAIN_FILE):
    non_target.append(img)


# In[ ]:


non_target


# In[ ]:


non_target_labels = np.c_[non_target,[0]*len(non_target)]
non_df = pd.DataFrame(non_target_labels,columns=['filename','label'])
non_df.head()


# In[ ]:


# imbalanced data visualization
import plotly.graph_objects as go

P = len(df)
N = len(non_df)

layout = go.Layout(
    title='Imbalanced data'
)
def plot_pie_chart(labels, values):
    fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
    fig.show()

labels = ['positive', 'Negative']
values = [P,N]
plot_pie_chart(labels, values)


# In[ ]:


#Augmentation
from matplotlib import image as mpimg
non_target = []
IMAGE_SIZE = (40,40)
for img in non_df['filename']:

    img = cv.imread("/content/drive/MyDrive/mlearn/dataset/butterfly/train/"+img)
    img = cv.resize(img,IMAGE_SIZE)
    non_target.append(img/255.0)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of the ImageDataGenerator class
datagen = ImageDataGenerator(
    rotation_range=30,  # Randomly rotate images by 30 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 20% of the total width
    height_shift_range=0.2,  # Randomly shift images vertically by 20% of the total height
    shear_range=0.3,  # Apply shear transformation with a shear angle of 30 degrees
    zoom_range=0.3,  # Randomly zoom images by up to 30%
    horizontal_flip=True,  # Randomly flip images horizontally
    vertical_flip=False  # Do not perform vertical flips
)

# Example grayscale numpy array of images (3D)
images = non_target # Shape: (num_samples, height, width)

# Generate augmented grayscale images
augmented_images = []

for image in images:
    num_generated_images = 0

    while num_generated_images < 60:
        augmented_image = datagen.random_transform(image)
        augmented_images.append(augmented_image)
        num_generated_images += 1
# Convert augmented grayscale images back to a NumPy array
augmented_images = np.array(augmented_images)
non_target = np.squeeze(augmented_images)
print(non_target.shape)
plt.imshow(non_target[0])
plt.axis('off')
plt.show()


# In[ ]:


plt.imshow(non_target[1])
plt.axis('off')
plt.show()


# In[14]:


IMAGE_SIZE = (40,40)
IMAGE_SIZE_CLASS = (60,60)

train = []

# it is for classification model which we will se after detection process
train_class = []
train_labels = []

test = []
print("starting")

for img,label in zip(df_train['filename'],df_train['label_en']):
    img = cv.imread("/content/drive/MyDrive/mlearn/dataset/butterfly/train/"+img)
    det = cv.resize(img,IMAGE_SIZE)
    clas = cv.resize(img,IMAGE_SIZE_CLASS)
    train.append(det/255.0)
    train_class.append(clas/255.0)
    train_labels.append(label)


for img in df_test['filename']:
    img = cv.imread("/content/drive/MyDrive/mlearn/dataset/butterfly/train/"+img)
    img = cv.resize(img,IMAGE_SIZE)
    test.append(img/255.0)

plt.imshow(train[0])
plt.axis('off')
plt.show()


# In[23]:


images = list(train)+list(test)

# Generate augmented images
augmented_images_target = []
count=0
print("Started..")
for image in images:
    num_generated_images = 0
    print("image -",count)
    count=count+1
    while num_generated_images < 6:
        print("going..")
        augmented_image = datagen.random_transform(image)
        augmented_images_target.append(augmented_image)
        num_generated_images += 1

augmented_images_target = np.array(augmented_images_target)
target = np.squeeze(augmented_images_target)
print("Ended")
print(target.shape)

P = len(target)
N = len(non_target)

layout = go.Layout(
    title='Balanced data'
)
def plot_pie_chart(labels, values):
    fig = go.Figure(data=[go.Pie(labels=labels, values=values)],layout=layout)
    fig.show()

labels = ['positive', 'Negative']
values = [P,N]
plot_pie_chart(labels, values)


# In[24]:


X = np.array(list(target)+list(non_target))
y = np.array([1]*len(target)+[0]*len(list(non_target)))

X.shape,y.shape


# In[25]:


# training and testing data spliting
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y)
X_train.shape,X_test.shape,y_train.shape,y_test.shape


# In[26]:


# Building CNN
import tensorflow as tf
from tensorflow import keras


# In[27]:


# Define the CNN model
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(40, 40, 3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=6, batch_size=32)


# In[28]:


model.evaluate(X_test,y_test)


# In[29]:


tensors = tf.convert_to_tensor(np.array(X_test))
probabilities = model.predict(tensors)
threshold = 0.5
y_pred = (probabilities > threshold).astype(int)[:,0]
y_pred


# In[30]:


import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix,classification_report
from seaborn import heatmap as hm

cm = confusion_matrix(y_test,y_pred)
hm(cm,annot=True,fmt='')
plt.xlabel("preidcte")
plt.ylabel("actual")
plt.show()


# In[31]:


cr = classification_report(y_test,y_pred)
print(cr)


# In[32]:


#spliting images for classification

train_class_np = np.array(train_class)
labels_np = np.array(train_labels)

X_train,X_test,y_train,y_test = train_test_split(train_class_np,labels_np,stratify=labels_np)
X_train.shape,X_test.shape,y_train.shape,y_test.shape


# In[33]:


# Define the CNN model
model_classification = keras.Sequential()
model_classification.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(60, 60, 3)))
model_classification.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_classification.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model_classification.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_classification.add(keras.layers.Flatten())
model_classification.add(keras.layers.Dense(64, activation='relu'))
model_classification.add(keras.layers.Dense(len(np.unique(labels_np)), activation='softmax'))

# Compile the model
model_classification.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_classification.fit(X_train, y_train, epochs=6, batch_size=32)


# In[34]:


model_classification.evaluate(X_test,y_test)


# In[35]:


probabilities = model_classification.predict(np.array(X_test))
y_pred = [np.argmax(img) for img in probabilities]
print(y_pred)


# In[36]:


probabilities = model_classification.predict(np.array([X_test[2]]))
probabilities


# In[ ]:










In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
